In [ ]:
import re
import requests
import json
import pandas as pd

# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sea
# import plotly.graph_objs as go 
# from urllib.parse import urlencode

pd.options.display.max_colwidth = 130 # увеличим максимальную длину столбцов pandas

In [ ]:
!ls -1 data

In [ ]:
df = pd.read_excel('data/avito_2022-08-26_17-29.xlsx', index_col=None)
print(len(df))
df.sample(2)

---

In [ ]:
# df[['adr']].drop_duplicates().sort_values(by='adr').to_csv('adr.csv',index=False,)

In [ ]:
# df[ df['adr'].str.match(r'.*[Аа]парт.*»ул.') ][['adr']].drop_duplicates()
# df[ df['adr'].str.match(r'.*ЖК.*»ул.') ][['adr']].drop_duplicates()

In [ ]:
# df['adr_'] = 
# ( 
#     df['adr']
#     .str.replace(r'.*[Аа]парт.*»ул.','ул.')
#     .str.replace(r'.*ЖК.*»ул.','ул.')
    

#     "Апарт-комплекс «GARDEN»ул. Лётчиков, д. 10"
#     "Дом по ул. Маринеско, 8аул. Александра Маринеско, д. 8А"
# "ЖК «Porto Franco»ул. Братьев Манганари, д. 20"

    
#     пр-т
#     Клубный комплекс
#     Клубный дом
#     Рекреационный комплекс «РАШАД»пр-д
    
#     садоводческое товарищество
#     садоводческое некоммерческое товарищество
#     товарищество собственников недвижимости
    
# )

---

In [ ]:
# avito.adr.unique()

In [ ]:
#Напишем фукнцию, которая добавит в адреса "город Севастополь" там, где его нет
def split_add(col):
    #Проверяем с помощью регулярного выражения вхождение последовательности "город Москва" в проверяемую строку
    if re.match('.*город Севастополь*',col):
        #Если входит, возвращаем как есть
        return col[:]
    #Если не входит, складываем последовательно строки и после возвращаем
    return 'город Севастополь' + ',' + col[:]

In [ ]:
split_add('ЖК «Доброгород»ул. Токарева, 7 этап, блок-секция 3')

In [ ]:
avito['adr'] = avito['adr'].apply(split_add)
avito.sample(5)

In [ ]:
map_adr = avito[['adr']].drop_duplicates().reset_index(drop=True)
map_adr['info'] = ''
map_adr

---

In [ ]:
from config import key

#запишем полученный ключ для геокодера
# key = '...'

# зададим параметры запроса, чтобы получать в ответ на запрос json - файл
params = { # словарь с параметрами запроса
        "format" : "json", # определяем формат ответа на запрос
    }

#зададим функцию, которая на вход будет получать адрес и возвращать район
def mapmaker(address,key=key,params=params):
    
    #создадим url для запроса координат
    url = 'https://geocode-maps.yandex.ru/1.x/?apikey=' + key + '&geocode=' + address

    #получим координаты точки
    try:
        response = requests.get(url, params = params) #сделаем запрос к геокодеру
        coord = (
            json.loads(response.text)
            ['response']
            ['GeoObjectCollection']
            ['featureMember']
            [0]
            ['GeoObject']
            ['Point']
            ['pos']
        )
    except:
        coord = 'Не определен'
        
    return(coord)

In [ ]:
#Пример работы функции mapmaker
mapmaker('город Севастополь,ул. Генерала Жидилова, 4')

In [ ]:
# map_adr['info'] = avito[2:3]['adr'].apply(mapmaker)
map_adr.loc[0:2,'info'] = avito.loc[0:3]['adr'].apply(mapmaker)
map_adr.loc[0:2,:]

---

In [ ]:
# NOTE: лимит 1К запросов в сутки на один токен

In [ ]:
# from config import key1

map_adr.loc[3:995,'info'] = avito.loc[3:995]['adr'].apply(mapmaker)

#После отрабатывания функции записываем адреса и координаты в файл
map_addr_un.to_csv('map_addr_1.csv', encoding='utf-8', index=False)

In [ ]:
# from config import key2

map_adr[996:1192]['info'] = avito[996:1192]['adr'].apply(mapmaker)

#После отрабатывания функции записываем адреса и координаты в файл
map_addr_un.to_csv('map_addr_2.csv', encoding='utf-8', index=False)

In [ ]:
# from config import key3

map_adr[1993:]['info'] = avito[1993:]['adr'].apply(mapmaker)

#После отрабатывания функции записываем адреса и координаты в файл
map_addr_un.to_csv('map_addr_3.csv', encoding='utf-8', index=False)

In [ ]:
# map_adr